<a href="https://colab.research.google.com/github/plnu-biomechanics/kin6015/blob/main/notebooks/kin6015_lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://www.pointloma.edu/sites/default/files/styles/basic_page/public/images/PLNU_Biomechanics_Lab_green_yellowSD_HiRes.png" width=400>

## **KIN 6015 Biomechanical Basis of Human Movement**
Instructor: Arnel Aguinaldo, PhD

**Lab 5 Data Processing**

In this lab, throwing data was collected using markerless motion capture and an instrumented mound in 4 conditions: full, feet stationary, locked hips-shoulders, and kneeling. Inverse kinematics (IK) and kinetics via inverse dynamics were estimated using Visual3D. The data were then exported as text (*.txt) files and uploaded to the class repository in the lab's [GitHub](https://github.com/plnu-biomechanics).

To further process the data for this lab, follow the steps in this **Colab notebook**, which contains instructions and sample code on how to wrangle and analyze the data.


### Create your own Colab Notebook

1. Go to **File -> New notebook in Drive** to open a new notebook in your Python environment:<br>
<img src="https://raw.githubusercontent.com/plnu-biomechanics/kin6015/main/notebooks/images/file_notebook.png" width=450>

2. Rename your Colab notebook using this naming format: **lastname_group_lab#.ipynb** (e.g., "aguinaldo_targaryen_lab1.ipynb")
3. Click on the **+ Code** option above to insert a new code cell: <br>
<img src="https://raw.githubusercontent.com/plnu-biomechanics/kin6015/main/notebooks/images/addcode.png" width=280>

4. The data you will parse and analyze for this lab will be copied from the lab's GitHub and temporarily stored in your Colab's runtime directory, which can be accessed by clicking on the folder icon in the left menu:<br>
<img src="https://raw.githubusercontent.com/plnu-biomechanics/kin6015/main/notebooks/images/colab_folder.png" width=400>

5. Copy the following lines of code to import the packages needed for this analysis and to load the data files into your working directory. Be sure to update the `GROUP` variable with your group's name. **Note**: These files are "runtime" access only, meaning they are only temporarily stored in your working directory and show up when your notebook is in session. However, the following code cell allows you to clone the zipped files to the working directory each time it is executed.


In [1]:
import urllib.request
import zipfile
import os

# --------------------------------------------------
# STUDENT INPUT (edit only this line; case-sensitive)
# --------------------------------------------------
GROUP = "stark"   # e.g., "targaryen", "stark", "lannister", "martell", "greyjoy"

# --------------------------------------------------
# Configuration (do NOT edit below)
# These lines create a directory for this lab in your
# Colab working directory.
# --------------------------------------------------
zip_dir = "kin6015/lab5"
os.makedirs(zip_dir, exist_ok=True)

zip_filename = f"spring2026_lab5_{GROUP}.zip"

url = (
    "https://raw.githubusercontent.com/"
    "plnu-biomechanics/kin6015/main/"
    f"labs/{zip_filename}"
)

zip_path = os.path.join(zip_dir, zip_filename)

# --------------------------------------------------
# Download zip file
# --------------------------------------------------
urllib.request.urlretrieve(url, zip_path)

# --------------------------------------------------
# Extract contents from the zipped file
# --------------------------------------------------
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(zip_dir)

print("Extracted files in lab directory:")
print(os.listdir(zip_dir))


Extracted files in lab directory:
['Stark_Lab5_HipShoulderLock.txt', '__MACOSX', 'Stark_Lab5_FeetStationary.txt', 'Stark_Lab5_Full.txt', 'Stark_Lab5_Kneeling.txt', 'spring2026_lab5_stark.zip']


## 🔄 Data Parsing

You can add your own code below to parse the data needed for this lab by either using GenAI (e.g., Gemini, ChatGPT) or simply copying the prepared code below.

### ✋ GenAI prompt:
Parse the data from the text files for the discrete metrics in the columns 2-6 in row 6 (index 5). The column headings are found in row 2: BALL_RELEASE_SPEED_mph, Pelvis_to_Trunk_ms, Max_Elbow_Varus_Torque_Nm, Back_Leg_GRF_Nm, Lead_Leg_GRF_Nm. Be sure to organize them by throwing conditions, which are denoted in the filename as metadata after last `_` character before the `.txt` extension.

### ✅ Pre-compiled code:


In [5]:
import pandas as pd
import os
import re # Import the regular expression module
import numpy as np # Import numpy for np.nan

def parse_txt_file(filepath):
  """Reads a text file, extracts specific discrete metrics from row 6 (index 5)
  and assigns predefined column names.

  Args:
    filepath (str): The full path to the text file.

  Returns:
    pd.DataFrame: A DataFrame containing the extracted discrete data for a single trial.
  """
  with open(filepath, 'r') as file:
    lines = file.readlines()

  # Ensure there are at least 6 lines for data in row 6 (index 5)
  if len(lines) < 6:
      raise ValueError(f"File {os.path.basename(filepath)} has too few lines to parse correctly for discrete metrics.")

  # The row is row 6, which is lines[5].
  data_row_str = lines[5].strip()
  data_values_all = data_row_str.split('\t')

  # Determine 'condition' from filename first
  filename = os.path.basename(filepath)
  condition = 'unknown'
  if 'Full' in filename:
    condition = 'full'
  elif 'FeetStationary' in filename:
    condition = 'feet_stationary'
  elif 'HipShoulderLock' in filename:
    condition = 'locked_hips_shoulders'
  elif 'Kneeling' in filename:
    condition = 'kneeling'

  # Predefined column names for the output DataFrame
  column_names = [
      'BALL_RELEASE_SPEED_mph',
      'Pelvis_to_Trunk_ms',
      'Max_Elbow_Varus_Torque_Nm',
      'Back_Leg_GRF_Nm',
      'Lead_Leg_GRF_Nm'
  ]

  # Dictionary to store parsed values for the current file
  parsed_values = {}

  if condition == 'kneeling':
    # Kneeling files only have the first 3 data columns (corresponding to indices 1, 2, 3 of data_values_all)
    # Check if data_values_all has enough elements for these 3
    if len(data_values_all) < 4: # Need at least index 3 (0-indexed)
        raise ValueError(f"File {filename} (kneeling) does not have enough columns for the first 3 metrics in row 6 (index 5). Expected at least 4 split elements, got {len(data_values_all)}.")

    parsed_values[column_names[0]] = data_values_all[1]
    parsed_values[column_names[1]] = data_values_all[2]
    parsed_values[column_names[2]] = data_values_all[3]
    parsed_values[column_names[3]] = np.nan # Missing for kneeling condition
    parsed_values[column_names[4]] = np.nan # Missing for kneeling condition
  else:
    # Other conditions expect all 5 metrics (columns 2-6, i.e., indices 1-5 of data_values_all)
    # Check if data_values_all has enough elements for these 5
    if len(data_values_all) < 6: # Need at least index 5 (0-indexed)
        raise ValueError(f"File {filename} ({condition}) does not have enough columns to extract all 5 metrics (cols 2-6) in row 6 (index 5). Expected at least 6 split elements, got {len(data_values_all)}.")

    parsed_values[column_names[0]] = data_values_all[1]
    parsed_values[column_names[1]] = data_values_all[2]
    parsed_values[column_names[2]] = data_values_all[3]
    parsed_values[column_names[3]] = data_values_all[4]
    parsed_values[column_names[4]] = data_values_all[5]

  # Create a DataFrame from the dictionary
  df = pd.DataFrame([parsed_values])

  # Convert numeric columns to appropriate data types
  for col in df.columns:
    try:
      df[col] = pd.to_numeric(df[col])
    except ValueError:
      pass # Keep as string if not numeric

  df['condition'] = condition
  df['filename'] = filename

  return df


# 1. Get the list of .txt files from the working directory
zip_dir = "kin6015/lab5"
all_files = os.listdir(zip_dir)
txt_files = [f for f in all_files if f.endswith('.txt')]

# 2. Initialize an empty list to store DataFrames
allmetrics = []

# 3. Iterate through each .txt file and apply the parse_txt_file function
for filename in txt_files:
    filepath = os.path.join(zip_dir, filename)
    try:
        df = parse_txt_file(filepath)
        allmetrics.append(df)
    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Check if allmetrics is not empty before concatenating
if allmetrics:
    # 4. Concatenate all DataFrames in allmetrics into a single DataFrame
    combined_discretes_df = pd.concat(allmetrics, ignore_index=True)

    # 5. Create discretes_df by selecting the specified columns
    discretes_df = combined_discretes_df[[
            'filename',
            'condition',
            'BALL_RELEASE_SPEED_mph',
            'Pelvis_to_Trunk_ms',
            'Max_Elbow_Varus_Torque_Nm',
            'Back_Leg_GRF_Nm',
            'Lead_Leg_GRF_Nm'
        ]]

    # 6. Display the head of discretes_df
    print("\nHead of discretes_df:")
    print(discretes_df.head())
else:
    print("No dataframes were successfully parsed and added to allmetrics.")
    discretes_df = pd.DataFrame() # Initialize an empty DataFrame if no files were processed successfully



Head of discretes_df:
                         filename              condition  \
0  Stark_Lab5_HipShoulderLock.txt  locked_hips_shoulders   
1   Stark_Lab5_FeetStationary.txt        feet_stationary   
2             Stark_Lab5_Full.txt                   full   
3         Stark_Lab5_Kneeling.txt               kneeling   

   BALL_RELEASE_SPEED_mph  Pelvis_to_Trunk_ms  Max_Elbow_Varus_Torque_Nm  \
0                    41.9            -6.66666                   55.15063   
1                    65.0            -6.66667                  126.26793   
2                    71.4            13.33337                  132.84457   
3                    50.4            -6.66666                   96.09460   

   Back_Leg_GRF_Nm  Lead_Leg_GRF_Nm  
0        610.69495        394.08441  
1        898.33899       1112.17932  
2       1201.27673       1729.09155  
3              NaN              NaN  


### 👏 Complete the Analysis

After you've completed this lab's analysis, do the following:

1. Copy and paste the above plots to your lab report
2. Export the tables using the code in the following cell. You can download them from your runtime directory onto your own computer.
2. Share your Colab notebook with me
3. Copy the link to your notebook and submit it along with your report for this lab on Canvas

In [6]:
import os

# Construct the full file path for the CSV file
output_filepath = os.path.join(zip_dir, f"{GROUP}_discretes_for_analysis_lab5.csv")

# Export the DataFrame to CSV
discretes_df.to_csv(output_filepath, index=False)

# Print a confirmation message
print(f"'discretes_df' exported to '{output_filepath}'")


'discretes_df' exported to 'kin6015/lab5/stark_discretes_for_analysis_lab5.csv'
